# __Python 으로 구현하는 쿼리문 : SQL vs Pandas__
<br>

### Table of contents
1. [Select](#p4section1)
2. [Where](#p4section2)
    - [조건의 결합](#p4section21)
    - [NULL 확인](#p4section22)
3. [GROUP BY](#p4section3)
4. [JOIN](#p4section4)
    - [INNER JOIN](#p4section41)
    - [LEFT OUTER JOIN](#p4section42)
    - [RIGHT OUTER JOIN](#p4section43)
    - [FULL OUTER JOIN](#p4section44)
5. [UNION](#p4section5)
    - [그룹화 및 정렬](#p4section51)
6. [Etc](#p4section6)
7. [DELETE](#p4section7)

<br>

기본적인 SQL 구문을 Python Pandas 라이브러리를 이용하여 구현하는 방법에 대한 설명입니다.  
Python을 통해 데이터를 불러온 후 데이터에 대해 일반적인 프로그래밍, 알고리즘, 머신러닝 등을 적용하여 고도화된 작업의 수행이 가능합니다.  
쿼리문을 Python 코드에서 직접 실행시킬 수도 있지만, 몇 가지 라이브러리를 사용하면 프로그래밍 측면에서 더 우수한 코드의 작성이 가능합니다.  
> 기존에 없던 새로운 문법으로 정의된 것이 아니라  
Python의 기본 문법에 고도화된 행렬 연산 방법이 더해지고  
현대 프로그래밍의 기본 구조인 객체지향 방법으로 표현한 것입니다.   
기존 프로그래밍 언어들에서 크게 벗어나지 않으면 더 편리하게 구성된 형태입니다.  

Pandas 홈페이지 내 아래 문서를 기반으로 정리하였습니다.  
https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html   
(&nbsp; \* 사용 시스템에 따라 sql 문은 일부 달라질 수 있습니다)

<br>

In [1]:
# 기본 설정 : pandas 및 numpy 라이브러리 호출
import pandas as pd
import numpy as np

In [2]:
# 웹페이지에서 데이터를 불러옵니다.
url = 'https://raw.github.com/pandas-dev/pandas/master/pandas/tests/data/tips.csv'
tips = pd.read_csv(url) # tips는 앞에서 소개된 DB table과 대응되는 dataframe 데이터 형식입니다.
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


<br>

## __SELECT__ <a name="p4section1"></a>
데이터 살펴 보기
> SELECT total_bill, tip, smoker, time  
FROM tips  
LIMIT 5;

In [3]:
# []은 데이터에 대한 호출, () 은 함수 등에 대한 일반적인 표현 방식입니다
tips[['total_bill', 'tip', 'smoker', 'time']].head(5)

,total_bill,tip,smoker,time
0,16.99,1.01,No,Dinner
1,10.34,1.66,No,Dinner
2,21.01,3.50,No,Dinner
3,23.68,3.31,No,Dinner
4,24.59,3.61,No,Dinner


<br>

## __WHERE__ <a name="p4section2"></a>
데이터에 대한 조건 적용
> SELECT *  
FROM tips  
WHERE time = 'Dinner'  
LIMIT 5;

In [4]:
tips[tips['time'] == 'Dinner'].head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [5]:
# 조건을 별도의 변수로도 활용할 수 있습니다.
is_dinner = tips['time'] == 'Dinner'
print(is_dinner.value_counts())
tips[is_dinner].head(5)

True     176
False     68
Name: time, dtype: int64


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### __조건의 결합__ <a name="p4section21"></a>
> SELECT *  
FROM tips  
WHERE time = 'Dinner' AND tip > 5.00;

In [6]:
 tips[(tips['time'] == 'Dinner') & (tips['tip'] > 5.00)].head(5)

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4


> SELECT *  
FROM tips  
WHERE size >= 5 OR total_bill > 45;

In [7]:
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)].head(5)

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6


### __NULL 확인__ <a name="p4section22"></a>

In [8]:
# NULL 데이터 사전 생성
frame = pd.DataFrame({'col1': ['A', 'B', np.NaN, 'C', 'D'],
                      'col2': ['F', np.NaN, 'G', 'H', 'I']})
print(frame)

  col1 col2
0    A    F
1    B  NaN
2  NaN    G
3    C    H
4    D    I


> SELECT *  
FROM frame  
WHERE col2 IS NULL;

In [9]:
 frame[frame['col2'].isna()]

,col1,col2
1,B,NaN


> SELECT *  
FROM frame  
WHERE col1 IS NOT NULL;

In [10]:
frame[frame['col1'].notna()]

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I


<br>

## __GROUP BY__ <a name="p4section3"></a>

> SELECT sex, count(*)  
FROM tips  
GROUP BY sex;

In [11]:
tips.groupby('sex').size()

sex
Female     87
Male      157
dtype: int64

In [12]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,87,87,87,87,87,87
Male,157,157,157,157,157,157


In [13]:
tips.groupby('sex')['total_bill'].count()

sex
Female     87
Male      157
Name: total_bill, dtype: int64

> SELECT day, AVG(tip), COUNT(*)  
FROM tips  
GROUP BY day;

In [14]:
tips.groupby('day').agg({'tip': np.mean, 'day': np.size})

,tip,day
day,,
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76
Thur,2.771452,62


> SELECT smoker, day, COUNT(*), AVG(tip)  
FROM tips  
GROUP BY smoker, day;

In [15]:
tips.groupby(['smoker', 'day']).agg({'tip': [np.size, np.mean]})

tip          
             size      mean
smoker day                 
No     Fri    4.0  2.812500
       Sat   45.0  3.102889
       Sun   57.0  3.167895
       Thur  45.0  2.673778
Yes    Fri   15.0  2.714000
       Sat   42.0  2.875476
       Sun   19.0  3.516842
       Thur  17.0  3.030000

<br>

## __JOIN__ <a name="p4section4"></a>

In [16]:
# 먼저 두 개의 dataframe 를 만듭니다.
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})
df2 = pd.DataFrame({'key': ['B', 'D', 'D', 'E'], 'value': np.random.randn(4)})
print(df1)
print('\n', df2)

  key     value
0   A  1.434771
1   B -1.236964
2   C -0.443782
3   D -0.507951

   key     value
0   B -1.302682
1   D -0.155957
2   D  1.197795
3   E  0.836543


### __INNER JOIN__ <a name="p4section41"></a>
> SELECT *  
FROM df1  
INNER JOIN df2  
  ON df1.key = df2.key;

In [17]:
pd.merge(df1, df2, on='key')

,key,value_x,value_y
0,B,-1.236964,-1.302682
1,D,-0.507951,-0.155957
2,D,-0.507951,1.197795


### __LEFT OUTER JOIN__ <a name="p4section42"></a>
> SELECT *  
FROM df1  
LEFT OUTER JOIN df2  
  ON df1.key = df2.key;

In [18]:
pd.merge(df1, df2, on='key', how='left') # df1 key를 기준으로 join

,key,value_x,value_y
0,A,1.434771,NaN
1,B,-1.236964,-1.302682
2,C,-0.443782,NaN
3,D,-0.507951,-0.155957
4,D,-0.507951,1.197795


### __RIGHT OUTER JOIN__ <a name="p4section43"></a>
> SELECT *  
FROM df1  
RIGHT OUTER JOIN df2  
  ON df1.key = df2.key;

In [19]:
pd.merge(df1, df2, on='key', how='right') # df2 key를 기준으로 join

,key,value_x,value_y
0,B,-1.236964,-1.302682
1,D,-0.507951,-0.155957
2,D,-0.507951,1.197795
3,E,NaN,0.836543


### __FULL OUTER JOIN__ <a name="p4section44"></a>

> SELECT *  
FROM df1  
FULL OUTER JOIN df2  
  ON df1.key = df2.key;

In [20]:
pd.merge(df1, df2, on='key', how='outer')

,key,value_x,value_y
0,A,1.434771,NaN
1,B,-1.236964,-1.302682
2,C,-0.443782,NaN
3,D,-0.507951,-0.155957
4,D,-0.507951,1.197795
5,E,NaN,0.836543


<br>

## __UNION__ <a name="p4section5"></a>

In [21]:
df1 = pd.DataFrame({'city': ['Chicago', 'San Francisco', 'New York City'], 'rank': range(1, 4)})
df2 = pd.DataFrame({'city': ['Chicago', 'Boston', 'Los Angeles'], 'rank': [1, 4, 5]})
print(df1)
print('\n', df2)

            city  rank
0        Chicago     1
1  San Francisco     2
2  New York City     3

           city  rank
0      Chicago     1
1       Boston     4
2  Los Angeles     5


> SELECT city, rank  
FROM df1  
UNION ALL  
SELECT city, rank  
FROM df2;  
/* 결과   
         city  rank  
      Chicago     1  
San Francisco     2  
New York City     3  
      Chicago     1  
       Boston     4  
  Los Angeles     5  
*/

In [22]:
pd.concat([df1, df2])

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
0,Chicago,1
1,Boston,4
2,Los Angeles,5


> SELECT city, rank  
FROM df1  
UNION  
SELECT city, rank  
FROM df2;  
/*  중복 제거됨  
         city  rank  
      Chicago     1  
San Francisco     2  
New York City     3  
       Boston     4  
  Los Angeles     5  
*/

In [23]:
pd.concat([df1, df2]).drop_duplicates() # 두 개 컬럼 모두 중복인 경우

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
1,Boston,4
2,Los Angeles,5


<br>

## __Etc__ <a name="p4section6"></a>

### 정렬 후 5개 이후 10개의 item 확인

> SELECT * FROM tips  
ORDER BY tip DESC  
LIMIT 10 OFFSET 5;

In [24]:
tips.nlargest(10+5, columns='tip').tail(10)

,total_bill,tip,sex,smoker,day,time,size
183,23.17,6.50,Male,Yes,Sun,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
47,32.40,6.00,Male,No,Sun,Dinner,4
239,29.03,5.92,Male,No,Sat,Dinner,3
88,24.71,5.85,Male,No,Thur,Lunch,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2
44,30.40,5.60,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
85,34.83,5.17,Female,No,Thur,Lunch,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4


### __그룹화 및 정렬__ <a name="p4section51"></a>
day 로 분리 후 totla_bill 역순으로 정렬하여 rn <3 이하인 값들만 선택 수 day, rn 순으로 표시
> SELECT * FROM (  
  SELECT  
    t.*,  
    ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn  
  FROM tips t  
)  
WHERE rn < 3  
ORDER BY day, rn;  

In [25]:
(tips.assign(rn=tips.sort_values(['total_bill'], ascending=False)
             .groupby(['day']).cumcount() + 1).query('rn < 3').sort_values(['day','rn']))

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2


In [26]:
# rank 를 사용하는 것도 가능
(tips.assign(rnk=tips.groupby(['day'])['total_bill']
             .rank(method='first', ascending=False)).query('rnk < 3').sort_values(['day','rnk']))

,total_bill,tip,sex,smoker,day,time,size,rnk
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1.0
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2.0
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1.0
212,48.33,9.00,Male,No,Sat,Dinner,4,2.0
156,48.17,5.00,Male,No,Sun,Dinner,6,1.0
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2.0
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1.0
142,41.19,5.00,Male,No,Thur,Lunch,5,2.0


> SELECT * FROM (  
  SELECT  
    t.*,  
    RANK() OVER(PARTITION BY sex ORDER BY tip) AS rnk  
  FROM tips t  
  WHERE tip < 2  
)  
WHERE rnk < 3  
ORDER BY sex, rnk;  

In [27]:
(tips[tips['tip'] < 2].assign(rnk_min=tips.groupby(['sex'])['tip']
                              .rank(method='min')).query('rnk_min < 3').sort_values(['sex','rnk_min']))

,total_bill,tip,sex,smoker,day,time,size,rnk_min
67,3.07,1.00,Female,Yes,Sat,Dinner,1,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,1.0
237,32.83,1.17,Male,Yes,Sat,Dinner,2,2.0


<br>

## __UPDATE__<a name="p4section6"></a>
> UPDATE tips  
SET tip = tip*2  
WHERE tip < 2;  

In [28]:
tips.loc[tips['tip'] < 2, 'tip'] *= 2
print(tips.head(10))

   total_bill   tip     sex smoker  day    time  size
0       16.99  2.02  Female     No  Sun  Dinner     2
1       10.34  3.32    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4
5       25.29  4.71    Male     No  Sun  Dinner     4
6        8.77  2.00    Male     No  Sun  Dinner     2
7       26.88  3.12    Male     No  Sun  Dinner     4
8       15.04  3.92    Male     No  Sun  Dinner     2
9       14.78  3.23    Male     No  Sun  Dinner     2


<br>

## __DELETE__<a name="p4section7"></a>
> DELETE FROM tips  
WHERE tip > 9;

In [29]:
tips = tips.loc[tips['tip'] <= 9]
tips2 = tips.sort_values(by=['tip'], ascending=False)
print(tips2.head(5))
print(tips2.tail(5))

     total_bill   tip   sex smoker   day    time  size
212       48.33  9.00  Male     No   Sat  Dinner     4
23        39.42  7.58  Male     No   Sat  Dinner     4
59        48.27  6.73  Male     No   Sat  Dinner     4
141       34.30  6.70  Male     No  Thur   Lunch     6
183       23.17  6.50  Male    Yes   Sun  Dinner     4
     total_bill  tip     sex smoker  day    time  size
169       10.63  2.0  Female    Yes  Sat  Dinner     2
163       13.81  2.0    Male     No  Sun  Dinner     2
162       16.21  2.0  Female     No  Sun  Dinner     3
159       16.49  2.0    Male     No  Sun  Dinner     4
187       30.46  2.0    Male    Yes  Sun  Dinner     5


<br><br>

---
##### [이전 페이지](pandas_tutorial3.ipynb)
##### [다음 페이지(처음)](pandas_front.ipynb)